<a href="https://colab.research.google.com/github/LeeDongN/Medica_AI2/blob/main/Update_02_22_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code read a folder of data. After applying preprocessing (DC component removal), it will extract number of maximas specified by a number and the labels will be assigned through final_function. The data is then balanced by ADASYN and SMOTE oversampling methods. At the end, Multiple models are used as classification and regression approaches.
1)KNN, Decision Tree, Random Forest, Gradiant Boosting. The accuracies will be compared with confusion matrices at the end.

In [11]:
import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt
import os
import sklearn
from google.colab import drive
from scipy.signal import find_peaks
drive.mount('/content/drive')
path = '/content/drive/MyDrive/new_data/1stSeries_07.05.21'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# functions

#### importing first data function

In [ ]:
def first_data_import():
  # import the first_data
  # first_data making
  first_data = []
  first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_a = float(first_data_origin.columns[0])
  first_b = float(first_data_origin.columns[1])
  first_data.loc[0] = [first_a, first_b]
  first_data.columns=['Wave_length', 'Amplitude']
  first_data['label']=1.3
  first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
  first_np = np.ravel(first_np, order='F')
  first_np = np.array([first_np])

  return first_data, first_np

#### importing whole data and stack function

In [12]:
import os
import numpy as np
import pandas as pd

def stack_data(path):
    stack_np = None  # Initialize as None to properly concatenate later
    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)

            # Extract the first two columns and rename them
            data = data.iloc[:, :2]
            data.columns = ['Wave_length', 'Amplitude']

            # Stack the data as a single row
            temp = data.values.ravel(order='F')  # Convert DataFrame to NumPy array and flatten
            temp = temp.reshape(1, -1)  # Reshape to a 2D array with a single row

            # Concatenate to stack_np
            if stack_np is None:
                stack_np = temp
            else:
                stack_np = np.concatenate((stack_np, temp), axis=1)

    return stack_np


#### removing DC function

In [13]:
# Implementing a simple DC removal function
def remove_dc(signal):
    # Calculate the mean of the signal
    signal_mean = np.mean(signal)
    # Remove the mean from the signal
    dc_removed_signal = signal - signal_mean
    return dc_removed_signal

#### finding peaks function

In [14]:
# Find peaks
def detect_peaks(signal, threshold=None, min_distance=1, prominence=None):
    if threshold is None:
        # Set threshold as a percentage of the maximum peak
        max_peak = np.max(signal)
    peaks, _ = find_peaks(signal, height=threshold, distance=min_distance, prominence=prominence)
    return peaks

#### peak values extracting function

In [15]:
def extract_peaks(X, Y, peak_range):
  # pandas to numpy
  peak_X = X.to_numpy()
  peak_Y = Y.to_numpy()
  # extract the peak value index
  index = list(peak_Y).index(max(peak_Y))

  # temp saving
  extracted_x = []
  extracted_y = []

  # extract the values from highest peak, and surrounding peaks
  for k in range(peak_range):
    # index labeling
    i = k + 1
    p_temp_index = index + i
    m_temp_index = index - i
    # extract y
    extracted_y.append(peak_Y[p_temp_index])
    extracted_y.append(peak_Y[m_temp_index])
    # extract X
    extracted_x.append(peak_X[p_temp_index])
    extracted_x.append(peak_X[m_temp_index])

  # extrack the center(highest) value
  extracted_x.append(peak_X[index])
  extracted_y.append(peak_Y[index])

  return extracted_x, extracted_y

#Final Function for data with two labels

In [ ]:
def final_function(path, peak_range):
    extracted_data = []  # Container for extracted peak data
    label = []  # Container for labels

    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Assign label based on folder name
            label.append(1 if float(folder_name) > 1.36 else 0)

            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)
            data_origin = pd.read_csv(file_path, skiprows=24)

            # Restore the removed data (the first data)
            a = float(data_origin.columns[0])
            b = float(data_origin.columns[1])
            data.loc[0] = [a, b]
            data.columns = ['Wave_length', 'Amplitude']

            # Remove DC offset
            dc_removed_Y = remove_dc(data['Amplitude'])

            # Detect peaks
            min_distance = 100  # Set the minimum distance between peaks
            local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
            local_maxima_x = data['Wave_length'][local_maxima_indices]
            local_maxima_y = dc_removed_Y[local_maxima_indices]

            # Extract peaks within the specified range
            extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)

            # Concatenate and append to extracted_data
            temp = np.concatenate((extracted_x, extracted_y))
            extracted_data.append(temp)

    # Convert lists to numpy arrays
    extracted_np_data = np.array(extracted_data)
    label = np.array(label)

    return extracted_np_data, label


#Final Function for Data with 5 labels

In [16]:
def final_function(path, peak_range):
    extracted_data = []  # Container for extracted peak data
    label = []  # Container for labels

    folder_list = os.listdir(path)

    for folder_name in folder_list:
        temp_folder_list = os.listdir(os.path.join(path, folder_name))

        for file_name in temp_folder_list:
            # Assign label based on folder name range
            folder_value = float(folder_name)
            if folder_value == 1.3:
                label.append(0)
            elif 1.3 <= folder_value < 1.32:
                label.append(1)
            elif 1.32 <= folder_value < 1.38:
                label.append(2)
            elif 1.38 <= folder_value < 1.39:
                label.append(3)
            elif folder_value >= 1.39:
                label.append(4)

            # Read the data into a Pandas DataFrame
            file_path = os.path.join(path, folder_name, file_name)
            data = pd.read_csv(file_path, skiprows=24)
            data_origin = pd.read_csv(file_path, skiprows=24)

            # Restore the removed data (the first data)
            a = float(data_origin.columns[0])
            b = float(data_origin.columns[1])
            data.loc[0] = [a, b]
            data.columns = ['Wave_length', 'Amplitude']

            # Remove DC offset
            dc_removed_Y = remove_dc(data['Amplitude'])

            # Detect peaks
            min_distance = 100  # Set the minimum distance between peaks
            local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
            local_maxima_x = data['Wave_length'][local_maxima_indices]
            local_maxima_y = dc_removed_Y[local_maxima_indices]

            # Extract peaks within the specified range
            extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)

            # Concatenate and append to extracted_data
            temp = np.concatenate((extracted_x, extracted_y))
            extracted_data.append(temp)

    # Convert lists to numpy arrays
    extracted_np_data = np.array(extracted_data)
    label_np = np.array(label)

    return extracted_np_data, label_np


In [22]:
path = '/content/drive/MyDrive/new_data/1stSeries_07.05.21'
extracted_data, label = final_function(path,12)
np.save("/content/drive/MyDrive/newdata_1stSeries.npy", extracted_data)
np.save("/content/drive/MyDrive/5labels.npy", label)

# Test

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

# Specify the file path
data_path = "/content/drive/MyDrive/newdata_1stSeries.npy"
labels_path = "/content/drive/MyDrive/5labels.npy"

# Load the data from the file
data = np.load(data_path)
labels = np.load(labels_path)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=10)

# Train the KNN classifier
knn.fit(X_train, y_train)

# Evaluate the KNN classifier
accuracy = knn.score(X_test, y_test)

# Predict labels for the test set
y_pred = knn.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"-----------------------------------Accuracy before Oversampling-----------------------------------")
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate accuracy for each class
class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

for i, acc in enumerate(class_accuracy):
    print(f"Accuracy for class {i}: {acc:.2f}")


-----------------------------------Accuracy before Oversampling-----------------------------------
Accuracy: 0.8431372549019608
Confusion Matrix:
[[ 0  0  0  0  1]
 [ 1  0  0  0  0]
 [ 0  2 12  0  1]
 [ 0  0  1  2  0]
 [ 0  0  0  2 29]]
Accuracy for class 0: 0.00
Accuracy for class 1: 0.00
Accuracy for class 2: 0.80
Accuracy for class 3: 0.67
Accuracy for class 4: 0.94


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

# Apply SMOTE oversampling to the entire dataset
X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(data, labels)

# Display class distribution after SMOTE oversampling
print("Class distribution after SMOTE:", sorted(Counter(y_resampled_smote).items()))

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Display class distribution after ADASYN oversampling
print("Class distribution after ADASYN:", sorted(Counter(y_resampled_adasyn).items()))

# Split the SMOTE oversampled data into training and testing sets
X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42)

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize KNN classifier
knn = KNeighborsClassifier(n_neighbors=10)

# Train the KNN classifier with SMOTE oversampled data
knn.fit(X_train_smote, y_train_smote)

# Evaluate the KNN classifier with SMOTE oversampled test data
y_pred_smote = knn.predict(X_test_smote)
accuracy_smote = accuracy_score(y_test_smote, y_pred_smote)
print("Overall Accuracy with SMOTE:", accuracy_smote)

# Calculate the confusion matrix with SMOTE oversampled test data
conf_matrix_smote = confusion_matrix(y_test_smote, y_pred_smote)
print("Confusion Matrix with SMOTE:")
print(conf_matrix_smote)

# Calculate accuracy for each class with SMOTE
class_accuracy_smote = conf_matrix_smote.diagonal() / conf_matrix_smote.sum(axis=1)

for i, acc in enumerate(class_accuracy_smote):
    print(f"Accuracy for class {i}: {acc:.2f}")

# Train the KNN classifier with ADASYN oversampled data
knn.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the KNN classifier with ADASYN oversampled test data
y_pred_adasyn = knn.predict(X_test_adasyn)
accuracy_adasyn = accuracy_score(y_test_adasyn, y_pred_adasyn)
print("Overall Accuracy with ADASYN:", accuracy_adasyn)

# Calculate the confusion matrix with ADASYN oversampled test data
conf_matrix_adasyn = confusion_matrix(y_test_adasyn, y_pred_adasyn)
print("Confusion Matrix with ADASYN:")
print(conf_matrix_adasyn)

# Calculate accuracy for each class
class_accuracy_adasyn = conf_matrix_adasyn.diagonal() / conf_matrix_adasyn.sum(axis=1)

for i, acc in enumerate(class_accuracy):
    print(f"Accuracy for class {i}: {acc:.2f}")


Class distribution after SMOTE: [(0, 144), (1, 144), (2, 144), (3, 144), (4, 144)]
Class distribution after ADASYN: [(0, 148), (1, 144), (2, 145), (3, 144), (4, 144)]
Overall Accuracy with SMOTE: 0.8611111111111112
Confusion Matrix with SMOTE:
[[30  1  0  0  0]
 [ 9 15  0  0  0]
 [ 2  2 29  0  0]
 [ 1  0  0 23  0]
 [ 1  0  2  2 27]]
Accuracy for class 0: 0.97
Accuracy for class 1: 0.62
Accuracy for class 2: 0.88
Accuracy for class 3: 0.96
Accuracy for class 4: 0.84
Overall Accuracy with ADASYN: 0.8896551724137931
Confusion Matrix with ADASYN:
[[30  0  0  0  0]
 [ 5 24  0  0  0]
 [ 2  2 25  0  0]
 [ 0  0  1 22  0]
 [ 1  0  2  3 28]]
Accuracy for class 0: 0.00
Accuracy for class 1: 0.00
Accuracy for class 2: 0.80
Accuracy for class 3: 0.67
Accuracy for class 4: 0.94


In [65]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from imblearn.over_sampling import ADASYN
from collections import Counter

# Assuming you have your features in 'data' and labels/targets in 'labels'

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Display class distribution after ADASYN oversampling
print("Class distribution after ADASYN:", sorted(Counter(y_resampled_adasyn).items()))

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

# Train the Decision Tree Regressor with ADASYN oversampled data
dt_regressor.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Decision Tree Regressor with ADASYN oversampled test data
y_pred_adasyn_dt = dt_regressor.predict(X_test_adasyn)
r2_adasyn_dt = r2_score(y_test_adasyn, y_pred_adasyn_dt)
mse_adasyn_dt = mean_squared_error(y_test_adasyn, y_pred_adasyn_dt)
print("R-squared with Decision Tree Regressor:", r2_adasyn_dt)
print("Mean Squared Error with Decision Tree Regressor:", mse_adasyn_dt)


Class distribution after ADASYN: [(0, 148), (1, 144), (2, 145), (3, 144), (4, 144)]
R-squared with Decision Tree Regressor: 0.9675295592977428
Mean Squared Error with Decision Tree Regressor: 0.06896551724137931


R-squared with SMOTE: 0.9739659227706269
Mean Squared Error with SMOTE: 0.05423263888888889
R-squared with ADASYN: 0.9725121484235041
Mean Squared Error with ADASYN: 0.05838275862068966


#Ensemble Learning on Decision Tree

In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import ADASYN
from collections import Counter

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Display class distribution after ADASYN oversampling
print("Class distribution after ADASYN:", sorted(Counter(y_resampled_adasyn).items()))

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the Decision Tree classifier with ADASYN oversampled data
decision_tree_classifier.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Decision Tree classifier with ADASYN oversampled test data
y_pred_adasyn_decision_tree = decision_tree_classifier.predict(X_test_adasyn)
accuracy_adasyn_decision_tree = accuracy_score(y_test_adasyn, y_pred_adasyn_decision_tree)
print("Overall Accuracy with Decision Tree:", accuracy_adasyn_decision_tree)

# Calculate the confusion matrix with ADASYN oversampled test data
#conf_matrix_adasyn_decision_tree = confusion_matrix(y_test_adasyn, y_pred_adasyn_decision_tree)
#print("Confusion Matrix with ADASYN and Decision Tree:")
#print(conf_matrix_adasyn_decision_tree)

# Calculate accuracy for each class
class_accuracy_adasyn_decision_tree = conf_matrix_adasyn_decision_tree.diagonal() / conf_matrix_adasyn_decision_tree.sum(axis=1)

for i, acc in enumerate(class_accuracy_adasyn_decision_tree):
    print(f"Accuracy for class {i}: {acc:.2f}")


Class distribution after ADASYN: [(0, 148), (1, 144), (2, 145), (3, 144), (4, 144)]
Overall Accuracy with Decision Tree: 0.9448275862068966
Accuracy for class 0: 1.00
Accuracy for class 1: 1.00
Accuracy for class 2: 0.86
Accuracy for class 3: 1.00
Accuracy for class 4: 0.88


In [62]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import ADASYN
from collections import Counter

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize Gradient Boosting classifier
gbm_classifier = GradientBoostingClassifier(random_state=42)

# Train the Gradient Boosting classifier with ADASYN oversampled data
gbm_classifier.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Gradient Boosting classifier with ADASYN oversampled test data
y_pred_adasyn_gbm = gbm_classifier.predict(X_test_adasyn)
accuracy_adasyn_gbm = accuracy_score(y_test_adasyn, y_pred_adasyn_gbm)
print("Gradient Boosting Classifier:")
print("Overall Accuracy :", accuracy_adasyn_gbm)

# Calculate the confusion matrix with ADASYN oversampled test data
#conf_matrix_adasyn_gbm = confusion_matrix(y_test_adasyn, y_pred_adasyn_gbm)
#print("Confusion Matrix with ADASYN and Gradient Boosting:")
#print(conf_matrix_adasyn_gbm)

# Calculate accuracy for each class
class_accuracy_adasyn_gbm = conf_matrix_adasyn_gbm.diagonal() / conf_matrix_adasyn_gbm.sum(axis=1)

for i, acc in enumerate(class_accuracy_adasyn_gbm):
    print(f"Accuracy for class {i}: {acc:.2f}")


Gradient Boosting Classifier:
Overall Accuracy : 0.9655172413793104
Accuracy for class 0: 1.00
Accuracy for class 1: 0.97
Accuracy for class 2: 0.97
Accuracy for class 3: 1.00
Accuracy for class 4: 0.94


In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import ADASYN
from collections import Counter

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the Random Forest classifier with ADASYN oversampled data
rf_classifier.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Random Forest classifier with ADASYN oversampled test data
y_pred_adasyn_rf = rf_classifier.predict(X_test_adasyn)
accuracy_adasyn_rf = accuracy_score(y_test_adasyn, y_pred_adasyn_rf)
print("Overall Accuracy with Random Forest:", accuracy_adasyn_rf)

# Calculate the confusion matrix with ADASYN oversampled test data
#conf_matrix_adasyn_rf = confusion_matrix(y_test_adasyn, y_pred_adasyn_rf)
#print("Confusion Matrix with ADASYN and Random Forest:")
#print(conf_matrix_adasyn_rf)

# Calculate accuracy for each class
class_accuracy_adasyn_rf = conf_matrix_adasyn_rf.diagonal() / conf_matrix_adasyn_rf.sum(axis=1)

for i, acc in enumerate(class_accuracy_adasyn_rf):
    print(f"Accuracy for class {i}: {acc:.2f}")


Overall Accuracy with Random Forest: 0.9517241379310345
Accuracy for class 0: 1.00
Accuracy for class 1: 1.00
Accuracy for class 2: 1.00
Accuracy for class 3: 1.00
Accuracy for class 4: 0.91


In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Initialize Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)

# Train the Random Forest Regressor with SMOTE oversampled data
rf_reg.fit(X_train_smote, y_train_smote)

# Evaluate the Random Forest Regressor with SMOTE oversampled test data
y_pred_smote_rf = rf_reg.predict(X_test_smote)
r2_smote = r2_score(y_test_smote, y_pred_smote_rf)
mse_smote = mean_squared_error(y_test_smote, y_pred_smote_rf)
print("R-squared with SMOTE:", r2_smote)
print("Mean Squared Error with SMOTE:", mse_smote)

# Train the Random Forest Regressor with ADASYN oversampled data
rf_reg.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Random Forest Regressor with ADASYN oversampled test data
y_pred_adasyn_rf = rf_reg.predict(X_test_adasyn)
r2_adasyn = r2_score(y_test_adasyn, y_pred_adasyn_rf)
mse_adasyn = mean_squared_error(y_test_adasyn, y_pred_adasyn_rf)
print("R-squared with ADASYN:", r2_adasyn)
print("Mean Squared Error with ADASYN:", mse_adasyn)


R-squared with SMOTE: 0.9739659227706269
Mean Squared Error with SMOTE: 0.05423263888888889
R-squared with ADASYN: 0.9725121484235041
Mean Squared Error with ADASYN: 0.05838275862068966


In [72]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.metrics import mean_absolute_error

# Apply ADASYN oversampling to the entire dataset
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(data, labels)

# Split the ADASYN oversampled data into training and testing sets
X_train_adasyn, X_test_adasyn, y_train_adasyn, y_test_adasyn = train_test_split(X_resampled_adasyn, y_resampled_adasyn, test_size=0.2, random_state=42)

# Initialize Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(random_state=42)

# Train the Gradient Boosting Regressor with ADASYN oversampled data
gb_regressor.fit(X_train_adasyn, y_train_adasyn)

# Evaluate the Gradient Boosting Regressor with ADASYN oversampled test data
y_pred_adasyn_gb = gb_regressor.predict(X_test_adasyn)
r2_adasyn_gb = r2_score(y_test_adasyn, y_pred_adasyn_gb)
mse_adasyn_gb = mean_squared_error(y_test_adasyn, y_pred_adasyn_gb)
print("R-squared Boosting Regressor:", r2_adasyn_gb)
print("Mean Squared Error with Gradient Boosting Regressor:", round(mse_adasyn_gb,2))

# Calculate the Mean Absolute Error (MAE)
mae_adasyn_gb = mean_absolute_error(y_test_adasyn, y_pred_adasyn_gb)
print("Mean Absolute Error with Gradient Boosting Regressor:", round(mae_adasyn_gb,2))


R-squared Boosting Regressor: 0.9819240111841766
Mean Squared Error with Gradient Boosting Regressor: 0.04
Mean Absolute Error with Gradient Boosting Regressor: 0.07
